<a href="https://colab.research.google.com/github/aastha12/Imbalanced-Dataset-Classification/blob/master/Imbalanced_classification_with_SMOTE_and_CuML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Aim:

Classify credict card transaction as fraud or not using imbalanced dataset.

## Source:

https://www.kaggle.com/janiobachmann/credit-fraud-dealing-with-imbalanced-datasets/notebook
https://www.kaggle.com/rafjaa/resampling-strategies-for-imbalanced-datasets

## About the data:

1. Except for the transaction and amount we dont know what the other columns are (due to privacy reasons). The only thing we know, is that those columns that are unknown have been scaled already.
2. The description of the data says that all the features went through a PCA transformation (Dimensionality Reduction technique) (Except for time and amount).
3.  Keep in mind that in order to implement a PCA transformation features need to be previously scaled. (In this case, all the V features have been scaled or at least that is what we are assuming the people that develop the dataset did.)

## The metric trap:

One of the major issues that novice users fall into when dealing with unbalanced datasets relates to the metrics used to evaluate their model. Using simpler metrics like accuracy_score can be misleading. In a dataset with highly unbalanced classes, if the classifier always "predicts" the most common class without performing any analysis of the features, it will still have a high accuracy rate, obviously illusory.

## Confusion matrix:

An interesting way to evaluate the results is by means of a confusion matrix, which shows the correct and incorrect predictions for each class.

## Rapids setup:

Before importing the data, we need to install rapids to use the cuml library and make use of GPUs.

## Environment Sanity Check 

Click the _Runtime_ dropdown at the top of the page, then _Change Runtime Type_ and confirm the instance type is _GPU_.

Check the output of `!nvidia-smi` to make sure you've been allocated a Tesla T4, P4, or P100.

In [1]:
!nvidia-smi

Sat Sep 12 14:20:21 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#Setup:
Set up script installs
1. Install most recent Miniconda release compatible with Google Colab's Python install  (3.6.7)
1. removes incompatible files
1. Install RAPIDS 0.14 libraries including:
  1. cuDF
  1. cuML
  1. cuGraph
  1. cuSpatial
  1. cuSignal
  1. xgboost and dask-xgboost
1. Set necessary environment variables
1. Copy RAPIDS .so files into current working directory, a workaround for conda/colab interactions
1. If running v0.11 or higher, updates pyarrow library to 0.15.x.

In [2]:
# Install RAPIDS
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!bash rapidsai-csp-utils/colab/rapids-colab.sh stable

import sys, os

dist_package_index = sys.path.index('/usr/local/lib/python3.6/dist-packages')
sys.path = sys.path[:dist_package_index] + ['/usr/local/lib/python3.6/site-packages'] + sys.path[dist_package_index:]
sys.path
exec(open('rapidsai-csp-utils/colab/update_modules.py').read(), globals())

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 185 (delta 5), reused 0 (delta 0), pack-reused 171
Receiving objects: 100% (185/185), 57.14 KiB | 350.00 KiB/s, done.
Resolving deltas: 100% (67/67), done.
PLEASE READ
********************************************************************************************************
Changes:
1. IMPORTANT CHANGES: RAPIDS on Colab will be pegged to 0.14 Stable until further notice.
2. Default stable version is now 0.14.  Nightly will redirect to 0.14.
3. You can now declare your RAPIDSAI version as a CLI option and skip the user prompts (ex: '0.14' or '0.15', between 0.13 to 0.14, without the quotes): 
        "!bash rapidsai-csp-utils/colab/rapids-colab.sh <version/label>"
        Examples: '!bash rapidsai-csp-utils/colab/rapids-colab.sh 0.14', or '!bash rapidsai-csp-utils/colab/rapids-colab.sh stable', or '!bash ra

Now you can run code!

We will balance the dataset using SMOTE with ENN. Then we will use svc() from cuml library.

In [3]:
import pandas as pd 
import numpy as np
import cuml

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
df=pd.read_csv('/content/drive/My Drive/Colab Notebooks/creditcard.csv')

In [9]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [10]:
df.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,3.919560e-15,5.688174e-16,-8.769071e-15,2.782312e-15,-1.552563e-15,2.010663e-15,-1.694249e-15,-1.927028e-16,-3.137024e-15,1.768627e-15,9.170318e-16,-1.810658e-15,1.693438e-15,1.479045e-15,3.482336e-15,1.392007e-15,-7.528491e-16,4.328772e-16,9.049732e-16,5.085503e-16,1.537294e-16,7.959909e-16,5.367590e-16,4.458112e-15,1.453003e-15,1.699104e-15,-3.660161e-16,-1.206049e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,1.088850e+00,1.020713e+00,9.992014e-01,9.952742e-01,9.585956e-01,9.153160e-01,8.762529e-01,8.493371e-01,8.381762e-01,8.140405e-01,7.709250e-01,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,-2.458826e+01,-4.797473e+00,-1.868371e+01,-5.791881e+00,-1.921433e+01,-4.498945e+00,-1.412985e+01,-2.516280e+01,-9.498746e+00,-7.213527e+00,-5.449772e+01,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,-5.354257e-01,-7.624942e-01,-4.055715e-01,-6.485393e-01,-4.255740e-01,-5.828843e-01,-4.680368e-01,-4.837483e-01,-4.988498e-01,-4.562989e-01,-2.117214e-01,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,-9.291738e-02,-3.275735e-02,1.400326e-01,-1.356806e-02,5.060132e-02,4.807155e-02,6.641332e-02,-6.567575e-02,-3.636312e-03,3.734823e-03,-6.248109e-02,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,4.539234e-01,7.395934e-01,6.182380e-01,6.625050e-01,4.931498e-01,6.488208e-01,5.232963e-01,3.996750e-01,5.008067e-01,4.589494e-01,1.330408e-01,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,2.374514e+01,1.201891e+01,7.848392e+00,7.126883e+00,1.052677e+01,8.877742e+00,1.731511e+01,9.253526e+00,5.041069e+00,5.591971e+00,3.942090e+01,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [11]:
df.Class.value_counts(normalize=True)

0    0.998273
1    0.001727
Name: Class, dtype: float64

99% of the rows belong to class 0 and only 1% belong to class 1. This is highly imbalanced dataset.

## Missing Values

In [12]:
df.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

## Scaling

In [13]:
#columns that are left to scale (Amount and Time)
from sklearn.preprocessing import RobustScaler

# RobustScaler is less prone to outliers.

rob_scaler = RobustScaler()

df['Amount']=rob_scaler.fit_transform(df['Amount'].values.reshape(-1,1))
df['Time']=rob_scaler.fit_transform(df['Time'].values.reshape(-1,1))

## Train/test split

In [15]:
from sklearn.model_selection import train_test_split 

X=df.copy()
X.drop(['Class'],axis=1,inplace=True)
y=df['Class']

#stratify=data is split in a stratified fashion, using this as the class labels.

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,stratify=y,random_state=123)

In [16]:
print("Shape of training features:",X_train.shape)
print("Shape of training labels:",y_train.shape)
print("Shape of testing features:",X_test.shape)
print("Shape of testing labels:",y_test.shape)

Shape of training features: (227845, 30)
Shape of training labels: (227845,)
Shape of testing features: (56962, 30)
Shape of testing labels: (56962,)


## SMOTE with ENN

In [18]:
!pip install imbalanced-learn

     |████████████████████████████████| 167 kB 2.8 MB/s 


In [20]:
from imblearn.combine import SMOTEENN 

smen = SMOTEENN(random_state=12, sampling_strategy = 1.0)
X_train_smen, y_train_smen = smen.fit_sample(X_train, y_train)

## Modeling

In [34]:
y_train_smen.dtypes

dtype('int64')

In [43]:
from cuml.svm import SVC
from cuml import LogisticRegression
#from cuml.linear_model import LogisticRegression
from cuml.ensemble import RandomForestClassifier as cuRFC
from sklearn.model_selection import cross_val_score

LR=LogisticRegression() #does not have random state 
RF=cuRFC(seed=123)
SV=SVC() #does not have seed. Seed is only used if probability=True

#using cross validation to get training f1 score

LR_score=cross_val_score(LR,X_train_smen,y_train_smen.astype('float64'),cv=5,scoring='f1_weighted') #needs X and y to have shape dtype
RF_score=cross_val_score(RF,X_train_smen.astype('float32'),y_train_smen.astype('int32'),cv=5,scoring='f1_weighted') #only uses float32
SV_score=cross_val_score(SV,X_train_smen,y_train_smen,cv=5,scoring='f1_weighted')

print("Logistic Regression Cross Validation F1 score:",LR_score.mean())
print("Random Forest Cross Validation F1 score:",RF_score.mean())
print("Support Vector Cross Validation F1 score:",SV_score.mean())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: For reproducible results, n_streams==1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_seed is set
  


Logistic Regression Cross Validation F1 score: 0.9485680506407903
Random Forest Cross Validation F1 score: 0.9996676869645171
Support Vector Cross Validation F1 score: 0.9837699345828843


In [54]:
#Checking accuracy and classification report on training set before hyperparameter tuning
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

SV.fit(X_train_smen,y_train_smen)
training_predictions_SVM=SV.predict(X_train_smen)
print(classification_report(y_train_smen,training_predictions_SVM))
print ("Accuracy on Training set : ", accuracy_score(y_train_smen, training_predictions_SVM))
print ("Recall score on Training set : ", recall_score(y_train_smen, training_predictions_SVM,average='weighted'))  

              precision    recall  f1-score   support

           0       0.98      0.99      0.98    227101
           1       0.99      0.98      0.98    227451

    accuracy                           0.98    454552
   macro avg       0.98      0.98      0.98    454552
weighted avg       0.98      0.98      0.98    454552

Accuracy on Training set :  0.9806490786532674
Recall score on Training set :  0.9806490786532674


Random Forest and Support Vector have performed the best. Random Forestis prone to overfitting. We shall select Support Vector Machines and tune the hyperparameters to add regularization.

## Hyperparameter Tuning

In [45]:
!pip install scikit-optimize

     |████████████████████████████████| 101 kB 2.3 MB/s 


In [46]:
SV

SVC(handle=<cuml.common.handle.Handle object at 0x7f718065ea90>, C=1, kernel='rbf', degree=3, gamma='scale', coef0=0.0, tol=0.001, cache_size=200.0, max_iter=-1, nochange_steps=1000, verbose=2)

In [47]:
#Bayesian optimization over hyper parameters.

from skopt import BayesSearchCV

#The strength of the regularization is inversely proportional to C.
#reducing C to increase regularization and avoid overfitting

tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-2, 1e-3, 1e-4],
                     'C': [0.001, 0.10, 0.0001]
                    }, 
                    {'kernel': ['sigmoid'], 'gamma': [1e-2, 1e-3, 1e-4],
                     'C': [0.001, 0.10, 0.0001]
                    },
                    {'kernel': ['linear'], 'C': [0.001, 0.10, 0.0001]
                    }
                   ]

tuned_SVM=BayesSearchCV(SV,tuned_parameters,cv=5,scoring='f1_weighted',random_state=12,verbose=15,n_jobs=-1)


In [48]:
#below lines will take a long time to run 
%time tuned_SVM.fit(X_train_smen,y_train_smen)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   50.2s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   50.6s
[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed:  1.4min remaining:   57.3s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.7min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.7min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   44.1s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   44.1s
[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed:  1.3min remaining:   53.1s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.6min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.6min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 5 folds for each of 1 candidates, totalling 5 fits


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   14.4s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   14.5s
[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed:   26.9s remaining:   18.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   33.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   33.0s finished
/usr/local/lib/python3.6/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   18.9s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   19.0s
[Parallel(n_jobs=-1)]: Done   3 out of   5 | elapsed:   34.5s remaining:   23.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   41.9s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   41.9s finished


CPU times: user 3min 55s, sys: 2min 28s, total: 6min 24s
Wall time: 1h 56min 36s


BayesSearchCV(cv=5,
              estimator=SVC(handle=<cuml.common.handle.Handle object at 0x7f718065ea90>, C=1, kernel='rbf', degree=3, gamma='scale', coef0=0.0, tol=0.001, cache_size=200.0, max_iter=-1, nochange_steps=1000, verbose=2),
              n_jobs=-1, random_state=12, scoring='f1_weighted',
              search_spaces=[{'C': [0.001, 0.1, 0.0001],
                              'gamma': [0.01, 0.001, 0.0001],
                              'kernel': ['rbf']},
                             {'C': [0.001, 0.1, 0.0001],
                              'gamma': [0.01, 0.001, 0.0001],
                              'kernel': ['sigmoid']},
                             {'C': [0.001, 0.1, 0.0001], 'kernel': ['linear']}],
              verbose=15)

In [50]:
# Print the tuned parameters and score
print("Tuned SVR Parameters: {}".format(tuned_SVM.best_params_))
print("Best F1 score score is {}".format(tuned_SVM.best_score_))

Tuned SVR Parameters: OrderedDict([('C', 0.1), ('gamma', 0.01), ('kernel', 'rbf')])
Best F1 score score is 0.9844861329468249


In [56]:
#Checking accuracy and classification report on training set
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

training_predictions=tuned_SVM.predict(X_train_smen)
print(classification_report(y_train_smen,training_predictions))
print ("Accuracy on Training set : ", accuracy_score(y_train_smen, training_predictions))
print ("Recall score on Training set : ", recall_score(y_train_smen, training_predictions,average='weighted'))  
print("ROC AUC score on Training set:",roc_auc_score(y_train_smen,training_predictions,average='weighted'))

              precision    recall  f1-score   support

           0       0.98      0.99      0.98    227101
           1       0.99      0.98      0.98    227451

    accuracy                           0.98    454552
   macro avg       0.98      0.98      0.98    454552
weighted avg       0.98      0.98      0.98    454552

Accuracy on Training set :  0.9833682394973512
Recall score on Training set :  0.9833682394973512
ROC AUC score on Training set: 0.9833709230206855


Recall score improved and accuracy also improved by a bit with hyperparameter tuning. Now, let's finally test it on the testing set.

## Test Set

In [57]:
yhat=tuned_SVM.predict(X_test)
print(classification_report(y_test,yhat))
print ("Accuracy on Testing set : ", accuracy_score(y_test, yhat)) 
print ("Recall score on Testing set : ", recall_score(y_test, yhat,average='weighted'))  
print("ROC AUC score on Testing set:",roc_auc_score(y_test,yhat,average='weighted'))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99     56864
           1       0.09      0.84      0.16        98

    accuracy                           0.98     56962
   macro avg       0.54      0.91      0.57     56962
weighted avg       1.00      0.98      0.99     56962

Accuracy on Testing set :  0.9845159931182191
Recall score on Testing set :  0.9845159931182191
ROC AUC score on Testing set: 0.9107526874002273


ROC AUC weighted score is 91% and weighted F1 score is 99%. Accuracy is 98%.